In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import networkx as nx
import ast
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tripadvisor-european-restaurants/tripadvisor_european_restaurants.csv


In [3]:
resturant_file="/kaggle/input/tripadvisor-european-restaurants/tripadvisor_european_restaurants.csv"
resturant_data=pd.read_csv(resturant_file)

/tmp/ipykernel_31/3687186386.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  resturant_data=pd.read_csv(resturant_file)


In [4]:
resturant_data.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,restaurant_link,restaurant_name,original_location,country,region,province,city,address,latitude,longitude,...,excellent,very_good,average,poor,terrible,food,service,value,atmosphere,keywords
0,g10001637-d10002227,Le 147,"[""Europe"", ""France"", ""Nouvelle-Aquitaine"", ""Ha...",France,Nouvelle-Aquitaine,Haute-Vienne,Saint-Jouvent,"10 Maison Neuve, 87510 Saint-Jouvent France",45.961674,1.169131,...,2.0,0.0,0.0,0.0,0.0,4.0,4.5,4.0,NaN,NaN
1,g10001637-d14975787,Le Saint Jouvent,"[""Europe"", ""France"", ""Nouvelle-Aquitaine"", ""Ha...",France,Nouvelle-Aquitaine,Haute-Vienne,Saint-Jouvent,"16 Place de l Eglise, 87510 Saint-Jouvent France",45.957040,1.205480,...,2.0,2.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,g10002858-d4586832,Au Bout du Pont,"[""Europe"", ""France"", ""Centre-Val de Loire"", ""B...",France,Centre-Val de Loire,Berry,Rivarennes,"2 rue des Dames, 36800 Rivarennes France",46.635895,1.386133,...,3.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,g10002986-d3510044,Le Relais de Naiade,"[""Europe"", ""France"", ""Nouvelle-Aquitaine"", ""Co...",France,Nouvelle-Aquitaine,Correze,Lacelle,"9 avenue Porte de la Correze 19170, 19170 Lace...",45.642610,1.824460,...,1.0,0.0,0.0,0.0,0.0,4.5,4.5,4.5,NaN,NaN
4,g10022428-d9767191,Relais Du MontSeigne,"[""Europe"", ""France"", ""Occitanie"", ""Aveyron"", ""...",France,Occitanie,Aveyron,Saint-Laurent-de-Levezou,"route du Montseigne, 12620 Saint-Laurent-de-Le...",44.208860,2.960470,...,4.0,7.0,0.0,0.0,0.0,4.5,4.5,4.5,NaN,NaN


## Cleaning and Preprocessing

### because of high nan we will clean data as per task so that we can use maximum data for our tasks

on this link i read about how and what kind of nan i need to do 

- https://datascience.stackexchange.com/questions/116622/what-should-you-do-with-nan-values
- https://www.reddit.com/r/MachineLearning/comments/1f0zq8l/p_dealing_with_large_tabular_dataset_with_lot_of/

In [5]:


# Print columns
print("Columns in dataset:", resturant_data.columns)

# Calculate NaN percentage before imputation
nan_percent_before = resturant_data.isna().mean() * 100
# print(f"before data {nan_percent_before.sort_values(ascending=False)}")

# Fill NaN values
resturant_data['cuisines'] = resturant_data['cuisines'].fillna('[]')
resturant_data['special_diets'] = resturant_data['special_diets'].fillna('[]')
resturant_data['features'] = resturant_data['features'].fillna('[]')
resturant_data['keywords'] = resturant_data['keywords'].fillna('')
resturant_data['vegetarian_friendly'] = resturant_data['vegetarian_friendly'].fillna('No')
resturant_data['vegan_options'] = resturant_data['vegan_options'].fillna('No')
resturant_data['gluten_free'] = resturant_data['gluten_free'].fillna('No')
resturant_data['avg_rating'] = resturant_data['avg_rating'].fillna(0)
resturant_data['total_reviews_count'] = resturant_data['total_reviews_count'].fillna(0)
resturant_data['city'] = resturant_data['city'].fillna('Unknown')
resturant_data['country'] = resturant_data['country'].fillna('Unknown')

# Parse list-like columns (convert string '[]' to actual lists)
def parse_list(x):
    if isinstance(x, str) and x.startswith('['):
        return ast.literal_eval(x)
    return x if isinstance(x, list) else []
    
resturant_data['cuisines'] = resturant_data['cuisines'].apply(parse_list)
resturant_data['special_diets'] = resturant_data['special_diets'].apply(parse_list)
resturant_data['features'] = resturant_data['features'].apply(parse_list)

# Calculate NaN percentage after imputation
nan_percent_after = resturant_data.isna().mean() * 100
# print(f"before data {nan_percent_after.sort_values(ascending=False)}")

# Print before and after NaN percentages
print("\nNaN Percentage Comparison:")
print(f"{'Column':<30} {'Before (%)':>22} {'After (%)':>22}")
print("-" * 55)
for col in resturant_data.columns:
    print(f"{col:<30} {nan_percent_before[col]:>12.2f} {nan_percent_after[col]:>12.2f}")

Columns in dataset: Index(['restaurant_link', 'restaurant_name', 'original_location', 'country',
       'region', 'province', 'city', 'address', 'latitude', 'longitude',
       'claimed', 'awards', 'popularity_detailed', 'popularity_generic',
       'top_tags', 'price_level', 'price_range', 'meals', 'cuisines',
       'special_diets', 'features', 'vegetarian_friendly', 'vegan_options',
       'gluten_free', 'original_open_hours', 'open_days_per_week',
       'open_hours_per_week', 'working_shifts_per_week', 'avg_rating',
       'total_reviews_count', 'default_language',
       'reviews_count_in_default_language', 'excellent', 'very_good',
       'average', 'poor', 'terrible', 'food', 'service', 'value', 'atmosphere',
       'keywords'],
      dtype='object')

NaN Percentage Comparison:
Column                                     Before (%)              After (%)
-------------------------------------------------------
restaurant_link                        0.00         0.00
restaurant_na

# 1.
- Graph Construction

  - Create an undirected graph where nodes are restaurants.

  - Connect two nodes if they share at least one attribute: cuisine, special_diets, features, or are located in the same city.

  - Store edge weights based on the number of shared features.

In [6]:
# Create graph
G = nx.Graph()

# Add nodes using restaurant_name
for idx, row in resturant_data.iterrows():
    G.add_node(row['restaurant_name'], 
               city=row['city'], 
               cuisines=row['cuisines'], 
               special_diets=row['special_diets'], 
               features=row['features'])

# Verify
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Sample nodes: {list(G.nodes(data=True))[:5]}")

Number of nodes: 840914
Sample nodes: [('Le 147', {'city': 'Toulouse', 'cuisines': [], 'special_diets': [], 'features': []}), ('Le Saint Jouvent', {'city': 'Saint-Jouvent', 'cuisines': [], 'special_diets': [], 'features': []}), ('Au Bout du Pont', {'city': 'Rivarennes', 'cuisines': [], 'special_diets': [], 'features': []}), ('Le Relais de Naiade', {'city': 'Lacelle', 'cuisines': [], 'special_diets': [], 'features': []}), ('Relais Du MontSeigne', {'city': 'Saint-Laurent-de-Levezou', 'cuisines': [], 'special_diets': [], 'features': []})]
